In [1]:
from detr_file_manager import FileManager
from detr_data_source import DataSource
from detr_factory import DETRFactory
from detr_config import Config
from detr_file_manager import FileManager
from detr_model_evaluator import ModelEvaluator
from detr_model_loader import ModelLoader
from sklearn.model_selection import KFold

In [2]:
Config.LOGS_DIR = '../Otros/DETR_logs_data_augmentation/'
hyperparameters = ('D-DETR', 100, 6)
detr_factory = DETRFactory(*hyperparameters)
file_manager = FileManager(detr_factory)
data_source = DataSource(detr_factory, file_manager)
model_loader = ModelLoader(detr_factory, file_manager)

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [3]:
train_dataset = data_source.get_train_dataset()
kfold = KFold(n_splits=Config.FOLDS, shuffle=True, random_state=123456)
kfold_split = kfold.split(train_dataset.ids)
train_ids, valid_ids = kfold_split.__next__()
valid_dataset = data_source.get_valid_dataset()
train_loader = data_source.get_dataloader(train_dataset, train_ids)
valid_loader = data_source.get_dataloader(valid_dataset, valid_ids)


Loading Annotations from:  InBreast_Coco\train\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading Annotations from:  InBreast_Coco\train\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [4]:
file_manager.set_testing_setup()
model = model_loader.load_best_model()
model_evaluator = ModelEvaluator(model, detr_factory)


Loading Model:  ../Otros/DETR_logs_data_augmentation/model=D-DETR_queries=100_layers=6\test\checkpoints\epoch=96-step=582.ckpt


In [7]:
# batch = next(iter(valid_loader))
# predictions = model_evaluator.generate_predictions(batch, threshold=0.2)
# predictions = model_evaluator._prepare_for_coco_detection(predictions)

In [5]:
import copy
coco_gt = copy.deepcopy(train_dataset.coco)
new_imgs = {k:v for k,v in coco_gt.imgs.items() if k in valid_ids}
coco_gt.imgs = new_imgs

In [6]:
model_evaluator.evaluate(coco_gt, valid_loader, threshold=0.001)

Evaluating on threshold:  0.001
Accumulating evaluation results...
DONE (t=0.03s).


{'Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]': 0.612,
 'Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ]': 0.852,
 'Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ]': 0.724,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]': 0.451,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]': 0.61,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]': 0.885,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]': 0.6,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]': 0.755,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]': 0.755,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]': 0.6,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]': 0.717,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]': 0.933}

In [9]:
# model_evaluator.evaluate(coco_gt, valid_loader, threshold=0.001)

Evaluating on threshold:  0.001
Accumulating evaluation results...
DONE (t=0.01s).


{'Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]': 0.073,
 'Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ]': 0.097,
 'Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ]': 0.091,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]': 0.151,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]': 0.02,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]': 0.303,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]': 0.2,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]': 0.227,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]': 0.227,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]': 0.3,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]': 0.167,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]': 0.3}

In [7]:
valid_dataset = copy.deepcopy(train_dataset)
valid_dataset.coco.imgs = {k:v for k,v in valid_dataset.coco.imgs.items() if k in valid_ids}
coco_gt = valid_dataset.coco

In [8]:
model_evaluator.evaluate(coco_gt, valid_loader, threshold=0.001)

Evaluating on threshold:  0.001
Accumulating evaluation results...
DONE (t=0.02s).


{'Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]': 0.612,
 'Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ]': 0.852,
 'Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ]': 0.724,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]': 0.451,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]': 0.61,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]': 0.885,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]': 0.6,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]': 0.755,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]': 0.755,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]': 0.6,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]': 0.717,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]': 0.933}